In [58]:
import nltk
#import advertools as adv
import json
import re
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix

In [2]:
#nltk.download_shell()

In [59]:
data = []
with open("/Users/ssvalipour/Downloads/text classification/English/english10k_tweets.json",encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [60]:
df=pd.DataFrame(columns=['tweet_id','tweet_text'])

In [18]:
print(df)

                                                           tweet_text
tweet_id                                                             
815331870015627265  Wind 1,4 m/s ENE. Barometer 1003,2 hPa, Fallin...
815332374909222912                               @pasdcheval i did it
815332387496357889       Happy New Year 2017! https://t.co/UFob5hzO5o
815332527254732800                         Happy new Year from sweden
815332597886840834                              Happy New Years!!!! 💫
815333076070965249  @nelson02051 @Arianna483 he is a hunk papi now...
815333369751990276                       @ThatManMatt Happy new year!
815333438374998016  @sarahchang Happy New Year and happy 300 years...
815333549289209856  Not only have the fireworks been going off sin...
815333756932333569  I know it probably didn't come out in January....
815333875077566465     when you p r o g r a m https://t.co/HXgSMwIpqs
815333969801728001   Happy New Years, bitches https://t.co/80VJ4w3TGt
815334054790893568  

In [61]:
for line in data:
    if "extended_tweet" in line:
        df = df.append({'tweet_id': line['id'],
            'tweet_text':line["extended_tweet"]['full_text']         
                       }, ignore_index=True)        
    else:
        df = df.append({'tweet_id': line['id'],
            'tweet_text':line['text']           
                       }, ignore_index=True)

In [62]:
df.set_index('tweet_id',inplace=True)

In [63]:
df_lable = pd.read_csv("en10k_markup.txt",sep=' ',index_col=1,names=["lable"])

In [64]:
df_final = pd.merge(df,df_lable, how='left',left_index=True,right_index=True)
df_final.fillna(0,inplace=True)

In [65]:
df_final.head()


,tweet_text,lable
tweet_id,,
815331870015627265,"Wind 1,4 m/s ENE. Barometer 1003,2 hPa, Fallin...",1.0
815332374909222912,@pasdcheval i did it,0.0
815332387496357889,Happy New Year 2017! https://t.co/UFob5hzO5o,0.0
815332527254732800,Happy new Year from sweden,0.0
815332597886840834,Happy New Years!!!! 💫,0.0


In [44]:
# DF TO EXCEL

writer = ExcelWriter('PythonSaeideh.xlsx')
df_final.to_excel(writer,'Sheet5')
writer.save()

# Porter stemmer


In [75]:
#stemmer = PorterStemmer(mode="MARTIN_EXTENSIONS")

In [77]:
#stemmer.stem('swimming')

# Snowball stemmer

In [66]:
stemmer = SnowballStemmer("english")

In [13]:
#stemmer.stem('swimming')

'swim'

In [67]:
def clean_tweet(tweet):
    tweet = re.sub('http\S+\s*', 'xurlx', tweet)  
    tweet = re.sub('\d+', 'xnumberx', tweet)  
    tweet = re.sub('#\S+', 'xhashtagx', tweet) 
    tweet = re.sub('@\S+', 'xuserx', tweet) 
    tweet = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', tweet)
    tweet = re.sub('\s+', ' ', tweet)  
    return map(lambda x:stemmer.stem(x),tweet.split())

In [68]:
df['tweet_text'].head(1000).apply(clean_tweet).apply(list)

tweet_id
815331870015627265    [wind, xnumberx, xnumberx, m, s, ene, baromet,...
815332374909222912                                 [xuserx, i, did, it]
815332387496357889                  [happi, new, year, xnumberx, xurlx]
815332527254732800                     [happi, new, year, from, sweden]
815332597886840834                                [happi, new, year, 💫]
815333076070965249     [xuserx, xuserx, he, is, a, hunk, papi, nowaday]
815333369751990276                           [xuserx, happi, new, year]
815333438374998016    [xuserx, happi, new, year, and, happi, xnumber...
815333549289209856    [not, onli, have, the, firework, been, go, off...
815333756932333569    [i, know, it, probabl, didn, t, come, out, in,...
815333875077566465              [when, you, p, r, o, g, r, a, m, xurlx]
815333969801728001                     [happi, new, year, bitch, xurlx]
815334054790893568                 [xuserx, same, to, you, amp, famili]
815334109656522753    [guess, who, been, on, stage, xus

In [49]:
# DF TO EXCEL

writer = ExcelWriter('PythonWithClean.xlsx')
df['tweet_text'].head(1000).apply(clean_tweet).apply(list).to_excel(writer,'Sheet1')
writer.save()

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict

In [77]:
X = df_final['tweet_text']
y = df_final['lable']

In [78]:
X.shape

(10000,)

# Naive Bayes classifier (NB)

In [79]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_tweet)),  
    ('classifier', MultinomialNB()),  
])

In [73]:
recall = cross_val_score(pipeline, X, y, cv=10,scoring='recall')
print(recall.mean())

0.9044896755162242


In [56]:
precision = cross_val_score(pipeline, X, y, cv=10,scoring='precision')
print(precision.mean())

0.9746900234634277


In [57]:
accuracy = cross_val_score(pipeline, X, y, cv=10,scoring='accuracy')
print(accuracy.mean())

0.9732007954007953


In [83]:
# Confusion matrix is not a score, it is a kind of summary of what happened during evaluation.

y_pred = cross_val_predict(pipeline, X , y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

#[row, column]
TP = conf_mat[1, 1]
TN = conf_mat[0, 0]
FP = conf_mat[0, 1]
FN = conf_mat[1, 0]

[[7696   53]
 [ 215 2036]]


In [ ]:
! [small confusion matrix](images?)

# Support Vector Machine(SVM)


In [18]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_tweet)),  
    ('classifier', SVC(gamma='scale')),  
])

In [19]:
recall = cross_val_score(pipeline, X, y, cv=10,scoring='recall')
print(recall.mean())

0.8831642084562439


In [20]:
precision = cross_val_score(pipeline, X, y, cv=10,scoring='precision')
print(precision.mean())

0.9984847454133169


# Random Forest(RF)

In [21]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_tweet)),  
    ('classifier', RandomForestClassifier(n_estimators=600)),  
])

In [22]:
recall = cross_val_score(pipeline, X, y, cv=10,scoring='recall')
print(recall.mean())

0.9662340216322518


In [23]:
precision = cross_val_score(pipeline, X, y, cv=10,scoring='precision')
print(precision.mean())

0.9986188136188137
